In [2]:
import logging
import sys
import requests
import json

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/.env', override=True)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [10]:
import os
apikey = os.environ.get('CORE_API_KEY')

In [7]:
apikey

In [11]:
def query_api(search_url, query, scrollId=None):
    headers={"Authorization": f"Bearer {apikey}"}
    if not scrollId:
        response = requests.get(f"{search_url}?q={query}&limit=100&scroll=true",headers=headers)
    else:
        response = requests.get(f"{search_url}?q={query}&limit=100&scrollId={scrollId}",headers=headers)
    print(response.content)        
    return response.json(), response.elapsed.total_seconds()

def scroll(search_url, query):
    allresults = []
    count = 0
    scrollId=None
    while True:
        result, elapsed =query_api(search_url, query, scrollId)
        scrollId=result["scrollId"]
        totalhits = result["totalHits"]
        result_size = len(result["results"])
        if result_size==0:
            break
        for hit in result["results"]:
            allresults.append(extract_info(hit))
        count+=result_size
        print(f"{count}/{totalhits} {elapsed}s")
    return allresults
        
def extract_info(hit):
    return {"id":hit["id"], "name": hit["name"], "url":hit["oaiPmhUrl"]}

In [12]:
results = scroll("https://api.core.ac.uk/v3/search/data-providers", "type:JOURNAL")

results

b'{"message":"The API key you provided is not valid."}'


KeyError: 'scrollId'

In [ ]:

def search_core_api(query):
    # CORE API endpoint for search
    url = "https://core.ac.uk:443/api-v2/articles/search"

    # Parameters for the API request
    params = {
        "apiKey": "YOUR_API_KEY",  # Replace with your CORE API key
        "title": query,
        "page": 1,
        "pageSize": 10  # Number of results to return
    }

    # Making the GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

# Example search
results = search_core_api("A. salmonicida subsp. masoucida")
print(json.dumps(results, indent=4))